In [1]:
import pandas as pd 
import numpy as np
import os
import re
import time

from datetime import timedelta
import datetime

In [2]:
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

In [41]:
file_list = os.listdir('./data/개별기업뉴스_크롤링/')
file_list

['AJ네트웍스.csv',
 'AK홀딩스.csv',
 'BGF.csv',
 'BGF리테일.csv',
 'BNK금융지주.csv',
 'BYC.csv',
 'CJ CGV.csv',
 'CJ.csv',
 'CJ대한통운.csv',
 'CJ씨푸드.csv',
 'CJ제일제당.csv',
 'CS홀딩스.csv',
 'DB.csv',
 'DB금융투자.csv',
 'DB손해보험.csv',
 'DB하이텍.csv',
 'DGB금융지주.csv',
 'DL.csv',
 'DL건설.csv',
 'DL이앤씨.csv',
 'DRB동일.csv',
 'DSR.csv',
 'DSR제강.csv',
 'E1.csv',
 'ESR켄달스퀘어리츠.csv',
 'F&F 홀딩스.csv',
 'F&F.csv',
 'GKL.csv',
 'GS.csv',
 'GS건설.csv',
 'GS글로벌.csv',
 'GS리테일.csv',
 'HDC.csv',
 'HDC랩스.csv',
 'HDC현대EP.csv',
 'HDC현대산업개발.csv',
 'HD현대.csv',
 'HJ중공업.csv',
 'HLB글로벌.csv',
 'HMM.csv',
 'HSD엔진.csv',
 'JB금융지주.csv',
 'JW생명과학.csv',
 'JW중외제약.csv',
 'JW홀딩스.csv',
 'KB금융.csv',
 'KC그린홀딩스.csv',
 'KC코트렐.csv',
 'KEC.csv',
 'KG스틸.csv',
 'KG케미칼.csv',
 'KH 필룩스.csv',
 'KISCO홀딩스.csv',
 'KPX케미칼.csv',
 'KPX홀딩스.csv',
 'KR모터스.csv',
 'KSS해운.csv',
 'KTcs.csv',
 'KTis.csv',
 'LF.csv',
 'LG.csv',
 'LG디스플레이.csv',
 'LG생활건강.csv',
 'LG에너지솔루션.csv',
 'LG유플러스.csv',
 'LG이노텍.csv',
 'LG전자.csv',
 'LG헬로비전.csv',
 'LG화학.csv',
 'LIG넥스원.csv',
 'LS.csv',
 'LS네트웍스.c

# 뉴스전처리 

## 날짜 바꿔주기 : 7일전 -> 마지막수정일 - 7일

In [42]:
# getctime : 생성날짜
# getmtime : 수정날짜
# getatime : 액세스날짜

In [43]:
os.path.getmtime('./data/개별기업뉴스_크롤링/AJ네트웍스.csv')

1654093218.9105725

In [44]:
mtime = time.ctime(os.path.getmtime('./data/개별기업뉴스_크롤링/노루홀딩스.csv'))
mtime

'Thu Jun  2 14:37:36 2022'

In [45]:
mtime1 = pd.to_datetime(mtime)
mtime1

Timestamp('2022-06-02 14:37:36')

In [46]:
mtime1.year

2022

In [47]:
df = pd.read_csv('./data/개별기업뉴스_크롤링/AJ네트웍스.csv',index_col=0)
df.columns = ['날짜','뉴스제목','뉴스요약']
df.head()

,날짜,뉴스제목,뉴스요약
0,7일 전,[특징주]AJ네트웍스 수익성 개선 전망에 5%대↑,AJ네트웍스가 외형성장과 자회사 구조조정 효과로 수익성이 개선될 것이란 전망에 장초...
1,2022.05.23.,"AJ네트웍스, 400억 단기 차입금 증가 결정",정보기술(IT) 기기 렌탈 기업인 AJ네트웍스는 회사채 상환과 운영자금으로 쓰기 ...
2,2022.05.16.,"AJ네트웍스, 1분기 영업익 192억원…전년比 399.2%↑",렌탈기업 AJ네트웍스는 연결재무제표 기준 지난 1분기 영업이익이 192억원을 기록해...
3,2022.05.10.,"[카드] AJ네트웍스, AJ교실숲 조성 프로젝트 실시",AJ네트웍스(095570)가 다양한 공기정화식물 배치를 통해 푸른 교실을 만드는 '...
4,7일 전,[투자심리과열 종목] AJ네트웍스ㆍ윌링스 상승...크로바하이텍ㆍ일진머티리얼...,"25일 주식시장에서 AJ네트웍스 주가는 전거래일 대비 3.90% 오른 7200원, ..."


In [48]:
df['날짜']

0              7일 전
1       2022.05.23.
2       2022.05.16.
3       2022.05.10.
4              7일 전
           ...     
2871    2020.07.20.
2872    2020.07.20.
2873    2020.07.20.
2874    2020.07.20.
2875    2020.07.20.
Name: 날짜, Length: 2876, dtype: object

In [49]:
# 일 전 빼주기 
# df['날짜1'] = df['날짜'].apply(lambda x : x.replace('일 전','') if len(x)<5 else x)

In [50]:
# 일 전 빼주기 
df['날짜'] = df['날짜'].apply(lambda x : mtime1 - pd.to_timedelta(int(x.replace('일 전','')), unit='D') if len(x)<5 else x)

In [51]:
df['날짜']

0       2022-05-26 14:37:36
1               2022.05.23.
2               2022.05.16.
3               2022.05.10.
4       2022-05-26 14:37:36
               ...         
2871            2020.07.20.
2872            2020.07.20.
2873            2020.07.20.
2874            2020.07.20.
2875            2020.07.20.
Name: 날짜, Length: 2876, dtype: object

In [52]:
df['날짜'] = pd.to_datetime(df['날짜'])

In [53]:
df['날짜'] = pd.to_datetime(df['날짜'])
df['날짜'] = df['날짜'].apply(lambda x : x.to_period(freq='D'))

In [54]:
df.head()

,날짜,뉴스제목,뉴스요약
0,2022-05-26,[특징주]AJ네트웍스 수익성 개선 전망에 5%대↑,AJ네트웍스가 외형성장과 자회사 구조조정 효과로 수익성이 개선될 것이란 전망에 장초...
1,2022-05-23,"AJ네트웍스, 400억 단기 차입금 증가 결정",정보기술(IT) 기기 렌탈 기업인 AJ네트웍스는 회사채 상환과 운영자금으로 쓰기 ...
2,2022-05-16,"AJ네트웍스, 1분기 영업익 192억원…전년比 399.2%↑",렌탈기업 AJ네트웍스는 연결재무제표 기준 지난 1분기 영업이익이 192억원을 기록해...
3,2022-05-10,"[카드] AJ네트웍스, AJ교실숲 조성 프로젝트 실시",AJ네트웍스(095570)가 다양한 공기정화식물 배치를 통해 푸른 교실을 만드는 '...
4,2022-05-26,[투자심리과열 종목] AJ네트웍스ㆍ윌링스 상승...크로바하이텍ㆍ일진머티리얼...,"25일 주식시장에서 AJ네트웍스 주가는 전거래일 대비 3.90% 오른 7200원, ..."


## 전처리: df_n

In [55]:
# 중복제거
df.drop_duplicates(inplace=True)
df

,날짜,뉴스제목,뉴스요약
0,2022-05-26,[특징주]AJ네트웍스 수익성 개선 전망에 5%대↑,AJ네트웍스가 외형성장과 자회사 구조조정 효과로 수익성이 개선될 것이란 전망에 장초...
1,2022-05-23,"AJ네트웍스, 400억 단기 차입금 증가 결정",정보기술(IT) 기기 렌탈 기업인 AJ네트웍스는 회사채 상환과 운영자금으로 쓰기 ...
2,2022-05-16,"AJ네트웍스, 1분기 영업익 192억원…전년比 399.2%↑",렌탈기업 AJ네트웍스는 연결재무제표 기준 지난 1분기 영업이익이 192억원을 기록해...
3,2022-05-10,"[카드] AJ네트웍스, AJ교실숲 조성 프로젝트 실시",AJ네트웍스(095570)가 다양한 공기정화식물 배치를 통해 푸른 교실을 만드는 '...
4,2022-05-26,[투자심리과열 종목] AJ네트웍스ㆍ윌링스 상승...크로바하이텍ㆍ일진머티리얼...,"25일 주식시장에서 AJ네트웍스 주가는 전거래일 대비 3.90% 오른 7200원, ..."
...,...,...,...
2856,2020-11-04,"SK케미칼, 상승흐름 전일대비 +5.03%... 기관 10,000주 순매수 중","35%), 부국철강(5.21%), 팬젠(8.7%), GV(10.17%), AJ네트웍..."
2868,2020-11-04,"오스코텍, 장시작 후 꾸준히 올라 +11.39%... 외국인 기관 동시 순매수 중","1%), 남화토건(10.87%), GV(12.14%), AJ네트웍스(22.35%),..."
2869,2020-11-04,"SK가스, 상승출발 후 현재 +5.14%... 이 시각 거래량 2만1369주",[그래프]SK가스 일봉 차트 [표] 거래원 상위(10:01 기준) 이 시각 거래소에...
2870,2020-11-04,"녹십자홀딩스, 전일대비 13.36% 상승중... 최근 주가 반등 흐름",[그래프]녹십자홀딩스 일봉 차트 [표] 거래원 상위(10:00 기준) 이 시각 거래...


In [56]:
# 회사이름 들어간 뉴스제목만 골라내기
df_n = df[df['뉴스제목'].str.contains('AJ네트웍스')]
df_n = df_n.sort_values(by='날짜')
df_n

,날짜,뉴스제목,뉴스요약
1313,2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스가 SK브로드밴드를 통해 진행중인 '스마트렌탈'의 위약금 규정 및 취급상...
1312,2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,..."
1311,2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,..."
1332,2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스(사장 반채운)가 파렛트와 고소장비 렌탈 부문의 신규 서비스 브랜드를 런...
1335,2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,신용등급이 BBB+인 AJ네트웍스도 회사채 발행을 위해 지난 20일 투자자들을 대상...
...,...,...,...
1,2022-05-23,"AJ네트웍스, 400억 단기 차입금 증가 결정",정보기술(IT) 기기 렌탈 기업인 AJ네트웍스는 회사채 상환과 운영자금으로 쓰기 ...
4,2022-05-26,[투자심리과열 종목] AJ네트웍스ㆍ윌링스 상승...크로바하이텍ㆍ일진머티리얼...,"25일 주식시장에서 AJ네트웍스 주가는 전거래일 대비 3.90% 오른 7200원, ..."
2757,2022-05-26,"[리포트 브리핑]AJ네트웍스, '렌탈 서비스의 강자, 사업 다각화에 성공. 커버...",로보뉴스 = 유진투자증권에서 25일 AJ네트웍스(095570)에 대해 '렌탈 서비스...
2758,2022-05-26,"[장중수급포착] AJ네트웍스, 외국인 35,548주 대량 순매수... 주가 +2.45%","로보뉴스 = 25일 10시 15분 현재 AJ네트웍스(095570)는 외국인이 35,..."


# 공시 전처리: df_a

In [3]:
df1 = pd.read_csv('./data/공시_다트크롤링/CJ.csv',index_col=0)
df1.head()

,번호,공시대상회사,보고서명,제출인,접수일자,비고
0,1,유 CJ,분기보고서 (2022.03),CJ,2022.05.16,NaN
1,2,유 CJ,기업설명회(IR)개최(안내공시),CJ,2022.05.09,유
2,3,유 CJ,현금ㆍ현물배당결정 (자회사의 주요경영사항),CJ,2022.05.09,유
3,4,유 CJ,[기재정정]현금ㆍ현물배당결정 (자회사의 주요경영사항),CJ,2022.04.06,유
4,5,유 CJ,사외이사의선임ㆍ해임또는중도퇴임에관한신고,CJ,2022.03.29,NaN


In [4]:
df1 = df1[['접수일자','공시대상회사','보고서명']]
df1.columns = ['날짜','공시회사','공시보고서명']
df1

,날짜,공시회사,공시보고서명
0,2022.05.16,유 CJ,분기보고서 (2022.03)
1,2022.05.09,유 CJ,기업설명회(IR)개최(안내공시)
2,2022.05.09,유 CJ,현금ㆍ현물배당결정 (자회사의 주요경영사항)
3,2022.04.06,유 CJ,[기재정정]현금ㆍ현물배당결정 (자회사의 주요경영사항)
4,2022.03.29,유 CJ,사외이사의선임ㆍ해임또는중도퇴임에관한신고
...,...,...,...
4,2016.02.04,유 CJ,매출액또는손익구조30%(대규모법인은15%)이상변동 (자회사의 주요경영사항)
5,2016.02.04,유 CJ,현금ㆍ현물배당결정 (자회사의 주요경영사항)
6,2016.02.03,유 CJ,풍문또는보도에대한해명
7,2016.02.02,유 CJ,현금ㆍ현물배당결정 (자회사의 주요경영사항)


In [5]:
df1['날짜'] = pd.to_datetime(df1['날짜'])
df1['날짜'] = df1['날짜'].apply(lambda x : x.to_period(freq='D'))

In [6]:
len(df1)

369

In [7]:
# 겹치는 공시
df1[df1.duplicated()]

,날짜,공시회사,공시보고서명
11,2022-03-21,유 CJ,감사보고서제출 (자회사의 주요경영사항)
0,2021-12-20,유 CJ,회사합병결정(자율공시)(종속회사의주요경영사항)
2,2021-12-17,유 CJ,특수관계인과의내부거래
3,2021-12-17,유 CJ,특수관계인과의내부거래
4,2021-12-17,유 CJ,특수관계인과의내부거래
5,2021-12-17,유 CJ,특수관계인과의내부거래
7,2020-12-28,유 CJ,특수관계인과의내부거래
8,2020-12-28,유 CJ,특수관계인과의내부거래
9,2020-12-28,유 CJ,특수관계인과의내부거래
10,2020-12-28,유 CJ,특수관계인과의내부거래


In [77]:
df1.drop_duplicates(subset=['공시보고서명']inplace=True)
df_a = df1.sort_values(by='날짜')
df_a

,날짜,공시회사,공시보고서명
5,2016-01-04,유 AJ네트웍스,임원ㆍ주요주주특정증권등소유상황보고서
4,2016-01-14,유 AJ네트웍스,타인에대한채무보증결정
3,2016-01-15,유 AJ네트웍스,[기재정정]타인에대한채무보증결정
2,2016-01-27,유 AJ네트웍스,[발행조건확정]증권신고서(채무증권)
1,2016-01-28,유 AJ네트웍스,투자설명서
...,...,...,...
4,2022-05-13,유 AJ네트웍스,분기보고서 (2022.03)
3,2022-05-18,유 AJ네트웍스,기업설명회(IR)개최(안내공시)
2,2022-05-23,유 AJ네트웍스,단기차입금증가결정
1,2022-05-23,유 AJ네트웍스,타인에대한채무보증결정


In [78]:
# 띄어쓰기 by PyKoSpacing

In [79]:
from pykospacing import Spacing

In [80]:
spacing = Spacing()
df_a['공시보고서명'] = df_a['공시보고서명'].apply(lambda x : spacing(x))
df_a

,날짜,공시회사,공시보고서명
5,2016-01-04,유 AJ네트웍스,임원 ㆍ 주요 주주 특정 증권 등 소유 상황 보고서
4,2016-01-14,유 AJ네트웍스,타인에 대한 채무보증 결정
3,2016-01-15,유 AJ네트웍스,[기재정정] 타인에 대한 채무보증 결정
2,2016-01-27,유 AJ네트웍스,[발행 조건 확정] 증권신고서(채무증권)
1,2016-01-28,유 AJ네트웍스,투자설명서
...,...,...,...
4,2022-05-13,유 AJ네트웍스,분기보고서 (2022.03)
3,2022-05-18,유 AJ네트웍스,기업설명회(IR) 개최(안내공시)
2,2022-05-23,유 AJ네트웍스,단기차입금 증가 결정
1,2022-05-23,유 AJ네트웍스,타인에 대한 채무보증 결정


In [76]:
# merge 
df_total = pd.merge(df_n1,df_a,on='날짜',how='outer')

In [74]:
# 겹치는 날짜 있는지 확인
df_total['날짜'].value_counts().nunique()

1

In [77]:
df_total.iloc[:,1:] = df_total.iloc[:,1:].fillna('')
df_total.sample(5)

,날짜,뉴스제목,뉴스요약,공시보고서명,공시회사
528,2021-07-15,"[스타워즈] 유안타證 홍광직, AJ네트웍스 등으로 누적수익률 90% 넘겨","그는 전일 AJ네트웍스 1200주를 평균 6159원에 샀다가, 장 종료 전 평균 6...",,
341,2019-10-23,10월 23일 AJ네트웍스 정보 -1.07% 변동폭 보여..주식증권 AJ네트웍스 주...,▲생활주가정보 AJ네트웍스 흐름▲AJ네트웍스 지난 한달간 흐름 ●주식종목차트분석 현...,,
541,2021-12-21,"AJ네트웍스 ""베트남 자회사에 240억원 출자""",코스피 상장사 AJ네트웍스는 베트남 냉동냉장창고 자회사에 240억원을 출자했다고 2...,단기차입금 증가 결정 타법인 주식 및 출자증권 취득 결정,유 AJ네트웍스유 AJ네트웍스
452,2020-07-13,"AJ네트웍스, +1.84% 상승폭 확대",AJ네트웍스(095570)가 상승폭을 확대하고 있다. 전일 대비 1.84% 상승한 ...,,
211,2019-04-15,AJ네트웍스 주가증권투자정보 및 주식시황 15일 현재 6390원,생활주가정보 AJ네트웍스 주가AJ네트웍스 지난 한달간 시세 ▶종목 그래프 분석 이 ...,,


In [81]:
df_total['뉴스요약']

0      AJ네트웍스가 SK브로드밴드를 통해 진행중인 '스마트렌탈'의 위약금 규정 및 취급상...
1      국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,...
2      국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,...
3      AJ네트웍스(사장 반채운)가 파렛트와 고소장비 렌탈 부문의 신규 서비스 브랜드를 런...
4      신용등급이 BBB+인 AJ네트웍스도 회사채 발행을 위해 지난 20일 투자자들을 대상...
                             ...                        
658                                                     
659                                                     
660                                                     
661                                                     
662                                                     
Name: 뉴스요약, Length: 663, dtype: object

In [82]:
# NaN = float형태여서 합칠 수 없음

df_total['뉴스제목+공시'] = df_total['뉴스제목']+ ' ' + df_total['공시보고서명']
# df_total['뉴스제목+요약+공시'] =  df_total['뉴스제목+공시']+ ' ' + df_total['뉴스요약'] 왜 안될까
df_total

,날짜,뉴스제목,뉴스요약,공시보고서명,공시회사,뉴스제목+공시
0,2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스가 SK브로드밴드를 통해 진행중인 '스마트렌탈'의 위약금 규정 및 취급상...,임원 ㆍ 주요 주주 특정 증권 등 소유 상황 보고서,유 AJ네트웍스,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄' 임원 ㆍ 주요 주주 특정 ..."
1,2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,...",,,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작"
2,2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,...",,,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작"
3,2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스(사장 반채운)가 파렛트와 고소장비 렌탈 부문의 신규 서비스 브랜드를 런...,,,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭"
4,2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,신용등급이 BBB+인 AJ네트웍스도 회사채 발행을 위해 지난 20일 투자자들을 대상...,,,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공
...,...,...,...,...,...,...
658,2022-03-18,,,[기재정정] 주주총회 소집 공고 주식 등의 대량 보유 상황 보고서(일반)[ 첨부 정...,유 AJ네트웍스유 AJ네트웍스유 AJ네트웍스,[기재정정] 주주총회 소집 공고 주식 등의 대량 보유 상황 보고서(일반)[ 첨부 ...
659,2022-03-21,,,감사보고서 제출사업보고서 (2021.12),유 AJ네트웍스유 AJ네트웍스,감사보고서 제출사업보고서 (2021.12)
660,2022-03-29,,,최대주주 등 소유 주식 변동신고서 주식매수선택권 부여에 관한 신고기업 설명회(IR)...,유 AJ네트웍스유 AJ네트웍스유 AJ네트웍스유 AJ네트웍스유 AJ네트웍스유...,최대주주 등 소유 주식 변동신고서 주식매수선택권 부여에 관한 신고기업 설명회(IR...
661,2022-05-18,,,기업설명회(IR) 개최(안내공시),유 AJ네트웍스,기업설명회(IR) 개최(안내공시)


In [47]:
# 에러났던 이유: NAN은 float형태라 문자열과 합칠 수 없음
# total_lst = []
# for i in range(len(df_total)):
#     ssum = df_total.iloc[i,1] + df_total.iloc[i,3]
#     total_lst.append(ssum)
    
# df_total['뉴스제목+공시'] = total_lst
# df_total

TypeError: can only concatenate str (not "float") to str

In [73]:
# 다른방법: str.cat()

# df_total['뉴스제목'].str.cat(df_total['공시보고서명'], sep=' ', na_rep='')

# 모델에 넣어줄 텍스트 데이터 전처리 함수:  text_data()

In [4]:
# 최고 성능의 모델 불러오기
BEST_MODEL_NAME = './data/model/best_model.h5'
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

In [5]:
def text_data(X_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments, targets = [], [], [], []
    
    for X in tqdm(X_data):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)
        
        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        
        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0]*MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)


    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)


    return [tokens, masks, segments]

# 공시에 감성점수 추가하기

In [97]:
df_a

,날짜,공시회사,공시보고서명
5,2016-01-04,유 AJ네트웍스,임원 ㆍ 주요 주주 특정 증권 등 소유 상황 보고서
4,2016-01-14,유 AJ네트웍스,타인에 대한 채무보증 결정
3,2016-01-15,유 AJ네트웍스,[기재정정] 타인에 대한 채무보증 결정
2,2016-01-27,유 AJ네트웍스,[발행 조건 확정] 증권신고서(채무증권)
1,2016-01-28,유 AJ네트웍스,투자설명서
...,...,...,...
4,2022-05-13,유 AJ네트웍스,분기보고서 (2022.03)
3,2022-05-18,유 AJ네트웍스,기업설명회(IR) 개최(안내공시)
2,2022-05-23,유 AJ네트웍스,단기차입금 증가 결정
1,2022-05-23,유 AJ네트웍스,타인에 대한 채무보증 결정


In [113]:
announce_data = df_a['공시보고서명']
len(announce_data)

263

In [114]:
announce_x = text_data(announce_data)
announce_x

100%|██████████████████████████████████████████████████████████████████████████████| 263/263 [00:00<00:00, 7324.28it/s]


[array([[   2, 6101,  200, ...,    0,    0,    0],
        [   2, 7122, 2170, ...,    0,    0,    0],
        [   2,   63, 7475, ...,    0,    0,    0],
        ...,
        [   2, 6010, 2232, ...,    0,    0,    0],
        [   2, 7122, 2170, ...,    0,    0,    0],
        [   2, 3646, 2068, ...,    0,    0,    0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [116]:
len(announce_x[0])

263

In [117]:
predicted_value = sentiment_model_best.predict(announce_x)
predicted_label = np.argmax(predicted_value, axis = 1)
predicted_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
      dtype=int64)

# 뉴스에 감성점수 추가하기

In [82]:
df_n.head()

,날짜,뉴스제목,뉴스요약
1313,2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스가 SK브로드밴드를 통해 진행중인 '스마트렌탈'의 위약금 규정 및 취급상...
1312,2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,..."
1311,2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,..."
1332,2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스(사장 반채운)가 파렛트와 고소장비 렌탈 부문의 신규 서비스 브랜드를 런...
1335,2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,신용등급이 BBB+인 AJ네트웍스도 회사채 발행을 위해 지난 20일 투자자들을 대상...


In [118]:
news_data = df_n['뉴스제목']
len(news_data)

907

In [119]:
news_x = text_data(news_data)
news_x

100%|██████████████████████████████████████████████████████████████████████████████| 907/907 [00:00<00:00, 4811.81it/s]


[array([[    2,    37,  2303, ...,     0,     0,     0],
        [    2,    37,  2303, ...,     0,     0,     0],
        [    2,    37,  2303, ...,     0,     0,     0],
        ...,
        [    2,    63, 15726, ...,     0,     0,     0],
        [    2,    63, 18487, ...,     0,     0,     0],
        [    2,    63,  4816, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [120]:
predicted_value = sentiment_model_best.predict(news_x)
predicted_label = np.argmax(predicted_value, axis = 1)
predicted_label

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2,
       2, 0, 0, 0, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 0, 0, 0, 2, 0, 2,
       0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 0, 2, 0, 1, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,

In [121]:
df_n['뉴스감성점수'] = predicted_label.tolist()
df_n

,날짜,뉴스제목,뉴스요약,뉴스감성점수
1313,2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스가 SK브로드밴드를 통해 진행중인 '스마트렌탈'의 위약금 규정 및 취급상...,0
1312,2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,...",0
1311,2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,...",0
1332,2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스(사장 반채운)가 파렛트와 고소장비 렌탈 부문의 신규 서비스 브랜드를 런...,0
1335,2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,신용등급이 BBB+인 AJ네트웍스도 회사채 발행을 위해 지난 20일 투자자들을 대상...,0
...,...,...,...,...
1,2022-05-23,"AJ네트웍스, 400억 단기 차입금 증가 결정",정보기술(IT) 기기 렌탈 기업인 AJ네트웍스는 회사채 상환과 운영자금으로 쓰기 ...,1
4,2022-05-26,[투자심리과열 종목] AJ네트웍스ㆍ윌링스 상승...크로바하이텍ㆍ일진머티리얼...,"25일 주식시장에서 AJ네트웍스 주가는 전거래일 대비 3.90% 오른 7200원, ...",0
2757,2022-05-26,"[리포트 브리핑]AJ네트웍스, '렌탈 서비스의 강자, 사업 다각화에 성공. 커버...",로보뉴스 = 유진투자증권에서 25일 AJ네트웍스(095570)에 대해 '렌탈 서비스...,1
2758,2022-05-26,"[장중수급포착] AJ네트웍스, 외국인 35,548주 대량 순매수... 주가 +2.45%","로보뉴스 = 25일 10시 15분 현재 AJ네트웍스(095570)는 외국인이 35,...",0


# (완성)뉴스: 날짜바꾸기 + 중복제거 합친코드

In [3]:
import time

In [12]:
# 날짜 바꾸기 합친코드

path = './data/개별기업뉴스_크롤링/'
file_list = os.listdir(path)

for file in file_list:
    
    # 파일 회사 이름(뒤 .csv 4자리 제거)
    corp_name = file[:-4]
    file_path = os.path.join(path,file)
    
    # 마지막 수정날짜
    mtime  = time.ctime(os.path.getmtime(file_path))
    mtime1 = pd.to_datetime(mtime)
    
    df = pd.read_csv(file_path,index_col=0)
    df.columns = ['날짜','뉴스제목','뉴스요약']
    
    # 3일 전 -> 마지막 수정날짜- 3일 로 바꾸기
    df['날짜'] = df['날짜'].apply(lambda x : mtime1 - pd.to_timedelta(int(x.replace('일 전','')), unit='D') \
                               if len(x)<5 else x)
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['날짜'] = df['날짜'].apply(lambda x : x.to_period(freq='D'))
    
    # 중복제거
    df.drop_duplicates(inplace=True)
    
    # 회사이름 들어간 뉴스제목만 골라내기
    df_n = df[df['뉴스제목'].str.contains(corp_name)]
    df_n = df_n.sort_values(by='날짜')
    
    df_n.to_csv(f'./data/개별기업뉴스_전처리/{corp_name}.csv')
    

In [13]:
# path = './data/개별기업뉴스_크롤링/'
# file_list = os.listdir(path)
# for file in file_list[:3]:
#     file_path = os.path.join(path,file)
# #     print(file)
#     corp_name = file[:-4]
#     print(corp_name)

## 누락기업 전처리

In [4]:
# 날짜 바꾸기 합친코드

path = './data/추가크롤링37/'
file_list = os.listdir(path)

for file in file_list:
    
    # 파일 회사 이름(뒤 .csv 4자리 제거)
    corp_name = file[:-4]
    file_path = os.path.join(path,file)
    
    # 마지막 수정날짜
    mtime  = time.ctime(os.path.getmtime(file_path))
    mtime1 = pd.to_datetime(mtime)
    
    df = pd.read_csv(file_path,index_col=0)
    df.columns = ['날짜','뉴스제목','뉴스요약']
    
    # 3일 전 -> 마지막 수정날짜- 3일 로 바꾸기
    df['날짜'] = df['날짜'].apply(lambda x : mtime1 - pd.to_timedelta(int(x.replace('일 전','')), unit='D') \
                               if len(x)<5 else x)
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['날짜'] = df['날짜'].apply(lambda x : x.to_period(freq='D'))
    
    # 중복제거
    df.drop_duplicates(inplace=True)
    
    # 회사이름 들어간 뉴스제목만 골라내기
    df_n = df[df['뉴스제목'].str.contains(corp_name)]
    df_n = df_n.sort_values(by='날짜')
    
    df_n.to_csv(f'./data/추가37_전처리/{corp_name}.csv')

## 용량적은거

In [9]:
# 날짜 바꾸기 합친코드

path = './data/temp1/'
file_list = os.listdir(path)

for file in file_list[4:5]:
    
    # 파일 회사 이름(뒤 .csv 4자리 제거)
    corp_name = file[:-4]
    file_path = os.path.join(path,file)
    
    # 마지막 수정날짜
    mtime  = time.ctime(os.path.getmtime(file_path))
    mtime1 = pd.to_datetime(mtime)
    
    df = pd.read_csv(file_path,index_col=0)
    df.columns = ['날짜','뉴스제목','뉴스요약']
    
    # 3일 전 -> 마지막 수정날짜- 3일 로 바꾸기
    df['날짜'] = df['날짜'].apply(lambda x : mtime1 - pd.to_timedelta(int(x.replace('일 전','')), unit='D') \
                               if len(x)<5 else x)
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['날짜'] = df['날짜'].apply(lambda x : x.to_period(freq='D'))
    
    # 중복제거
    df.drop_duplicates(inplace=True)
    
    # 회사이름 들어간 뉴스제목만 골라내기
    df_n = df[df['뉴스제목'].str.contains(corp_name)]
    df_n = df_n.sort_values(by='날짜')
    
    df_n.to_csv(f'./data/temp/{corp_name}.csv')